<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('data/online_shoppers_intention_DATAPTDIC19.csv', sep=';')

In [3]:
data.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0.0,0,0.0,0,1.0,0,0.20,0.20,0,0.0,Feb,1.0,1,1,1,Returning_Visitor,False,FALSE
1,0.0,0,0.0,0,2.0,64,0.00,0.10,0,0.0,Feb,2.0,2,1,2,Returning_Visitor,False,FALSE
2,0.0,-1,0.0,-1,1.0,-1,0.20,0.20,0,0.0,Feb,4.0,1,9,3,Returning_Visitor,False,FALSE
3,0.0,0,0.0,0,2.0,2.666.666.667,0.05,0.14,0,0.0,Feb,3.0,2,2,4,Returning_Visitor,False,FALSE
4,0.0,0,0.0,0,10.0,627.5,0.02,0.05,0,0.0,Feb,3.0,3,1,4,Returning_Visitor,True,FALSE
